# Some cryptography examples
A few simple examples of cryptographic functions in Python, primarily using the [cryptography](https://cryptography.io/) module.

In [64]:
import cryptography
import hashlib

from IPython.display import display

## Hashes
Create a hash - a fixed-length 'digest' - from a variable-length string input.  Hashing the same text will always produce the same hash.

In [35]:
text = 'In spite of all their friends could say, on a winter’s morn, on a stormy day, in a Sieve they went to sea!'
text = text.encode('utf-8') # encode the string
hash = hashlib.sha256( text ).hexdigest()
hash

'eab4c3a7b50af92be780ab19ba1e8764a4809c9bc7f3d2d966c0c24c7917b23f'

We can add in a **nonce** or a **salt** - additives that change the input such that it produces a different hash than the original input.  Here we hash the same input with `10` different nonces to show the difference.

In [107]:
# loop 10 times
for i in range( 10 ):
    # use the loop counter as the nonce
    nonce = str(i).encode('utf-8')
    # hash using the nonce
    hash = hashlib.sha256( text + nonce ).hexdigest()
    display( hash )

'331bf59c5f43ff421d728ed9608c5fb10149d40b044b97d4fc09d23dbfea998f'

'a99f2b8260bc2fa8405a3509ee75f74c79e169ad4bf45111563ac4e89d3a0cfe'

'6f19c2dabbbf49597278bdd007f9c4f88dd11d02b8cba7f7e882b87689a819a5'

'4a77691760dcc7706d8d91a6438d752b5d922c7f54dac0acc5e54369c3f46b9e'

'0931b7d7e8df4fa4c2a3e492dce77ff96cd204166b1bdf705d432a91e6c6d88e'

'2ada69ce9ff5284066eaca28f075e533d8ac5aae2398a95258c278df9dc30d9c'

'0c3cd5559f124136421776491ca8bf61a9d56d88bbca1a8bf55e9485e4205cbb'

'1836bd02f004e9c35adbf7f1a50441e6d2561b8eae8adf0aaf71c741efffa30e'

'02a87a2d316c16ff42b1c2b3dd63dc0feebbf0e496b4b4c5d25da8c85745d8f5'

'b0703807d662aecfdd940eac6cbd5245be49698d6d66f276444edad89e2274fe'

There is no way to predict what a hash of any input will be.  Let's try to find a hash that starts with two zeros, by brute force... this might take a while!

In [46]:
# loop indefinitely
i = 0
while True:
    # use the loop counter as the nonce
    nonce = str(i).encode('utf-8')
    # hash using the nonce
    hash = hashlib.sha256( text + nonce ).hexdigest()
    # display( hash )
    
    # check whether the hash starts two zeros
    if hash[0] == '00':
        display('Found it on the {}th iteration!'.format(i))
        display( hash )
        break
    
    # increment the counter
    i += 1

KeyboardInterrupt: 

## Create a private/public key pair
For public key cryptography, such as for signing a message to prove its authenticity or for sending an encrypted message to a recipient, we will need a private/public key pair.

In [108]:
# import the relevant parts of the cryptography module
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

In [109]:
private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048,
)
private_key

Derive a public key from the private key.

In [110]:
public_key = private_key.public_key() # derive the public key
public_key

## Digital signatures
Prove authenticity of a message by having the sender asign it with their private key, and the receipient validate the signature using the sender's public key.

### Create the signature
Each signature is unique, even if repeatedly signing the same message with the same private key.

In [123]:
# the message for most crypto functions must be a byte array, not a regular string
message = b"In spite of all their friends could say, on a winter's morn, on a stormy day, in a Sieve they went to sea!"

# generate a signature for this message using our private key
signature = private_key.sign(
    message,
    padding.PSS(
        mgf=padding.MGF1(hashes.SHA256()),
        salt_length=padding.PSS.MAX_LENGTH
    ),
    hashes.SHA256()
)
signature

b'I\x04\x0e4f\xd2\x986\xa4\xf8\xeb\xa2-6v\xaaM\xc2d$\x05>\xe6,\xf1\xe0\x18\x17\xef\x9a\xe2\xd8D\xbc\xdd\xc8\x89\xb2\xca\n\xb1\xbe\xe4\xef\xf4\xabP\xe5K\xc3\x862\xcb?"i\x19\x96\xfdq\xea\xee\x12\x96\xdfk`@\xdfp\x18\xf5\x18[\x15\'\xa7\xdb\x87Q6l\xbc\xf8\x8d}\x91\x15m\x93\xb7\xba\x0f\x15\x11&\x0c\xc69\xd7T0)\xec\xaea\xe6;\xb6\x93+h\x0ei\xdd\xc7\xf8TF\xd6#t]\x17\xb8\xef\xed\xc6\xfd\x98\xcc\xbb.\x868\xe1\x9c\x1a\\\r\x9c%\r$KW\x8c\x1d\x9c\x9b0\x1d\xf7\xbczN\xa4\xc5P\x04\xa6\xfa\x18i\x8d\x97@D&\x9e\xce\xf6Y\x17Cb\x0f\xd1\x95\xa2\xc7\x14\xe0\xc1\x1cY\xca\x1b\x80\x1f\xa7|\xbb\x1ds@\x97\x12aL\x9c=;\x90\x9f\xe8H\xd2\x87\x0c\x95gh\x89\xd4\xd4\xf9\xd2\xd6\n\xdf\xb4)8\xf1\xe4\x9b)\x142eN\x8f\xa6\x16\xe9\x91\x9c]\x89G\xd8\xf6W\xfbt\x80\x9c.\xb6X\xe8\x93\xd3.{'

### Validate the signature
Assuming they have been given the sender's public key, the recipient of this message could use the sender's public key to validate the authenticity of the message.

In [124]:
# If the signature does not match, verify() will raise an InvalidSignature exception.
try:
    # try to verify the signature
    public_key.verify(
        signature,
        message,
        padding.PSS(
            mgf=padding.MGF1(hashes.SHA256()),
            salt_length=padding.PSS.MAX_LENGTH
        ),
        hashes.SHA256()
    )
    display('The signature is valid.')
except:
    # an exception was thrown
    display('Invalid signature.  Be careful!')

'The signature is valid.'

## Encrypting messages
To send private messages that others will not be able to intercept, the sender can use the recipient's public key to encrypt a message in such a way that only the recipient (the holder of the corresponding private key) can decrypt it.

In [125]:
message = b"In spite of all their friends could say, on a winter's morn, on a stormy day, in a Sieve they went to sea!"

# create the encrypted message, a.k.a. the cyphertext
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# show the ciphertext
ciphertext

b'\x87\x80D\x05\x12\xf4P\x8a\xed15\xa1\xfb\x10\xf6\xcc\x9b{\x98\x89Q\x8e\xd3%B%!\xe9\xd2\xd0\xc6\x800\xffo\xe0\xb6\xa5D\xbd\x16\xc6\x8a>\x9b%O_.\xf3\xbc\xe8\xff\xa7\x9aa\xc1\xb0\xc0\xf1+\xb6<\x1c\xf4c\xc9b\x87\xd5\x95s!\xa4\xde\xb1E\xa6\xfd\x9dL\xa6\xa8\xaf\x1f\x0f\x1aI\xf1\xc4\xd1 %\x9bj\t\xbb#\xda\x0b\xb9\xf9\x97!-\x1d\x89dR\x95H\xff\xc4)\x01\x10#\x05\x83"\xc0\xed\xd4\xf7om3Y\x12\xed<\xe1\x8e\xecJ\x9f\x018a\x86\x9e\xa5\xcff\x1b\xef\xed\xeb\x9b|\x8a\x04\x9dz\x8c\xbeg\xcf\xf96~@6\x8d\xddZ\x83-\xae%\xa2\x0c\xc2\xd7\xf0\x82\xc7\xc0@,\xf88a\xe5V\x0b?\x10\x045\xad\xf9\xbf4\xb1\xf9\x9bk\x9f\xc9V\x18\xd8\'1 \xcc\x99$\x8b\xe2\x88-\x8f\xf5\xee5\x89\x04\x9c\xfez\xf6\xc28<\x96-\xd0\xca\xeaq\xfaRt\x16z=f\xb1\x9f\xc6_\x93r\xa7\xdf\x9c\x1a\x01Y\xe6#\x08a\xa9'

### Decrypt the message
The recipient can now use their own private key to decrypt the message that was encrypted by the sender specifically for them.  Imagine here that the recipient runs this code.

In [127]:
plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA256()),
        algorithm=hashes.SHA256(),
        label=None
    )
)

# for debugging purposes, we can verify that this decrypts the original message
plaintext == message

True

Assuming the decryption was successful, we can output the deciphered message:

In [128]:
plaintext

b"In spite of all their friends could say, on a winter's morn, on a stormy day, in a Sieve they went to sea!"